#### On my machine (Mike) I use conda env "trials"

In [318]:
from dotenv import load_dotenv
import os
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

#### Get protocol reference, vectorize and store in Qdrant

In [319]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Qdrant
from langchain_core.documents.base import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the document - here we are just using a protocol in a specific directory
file_path = './documents/protocol.pdf'
# file_path = './documents/consent.pdf'
separate_pages = []             
loader = PyMuPDFLoader(file_path)
page = loader.load()
separate_pages.extend(page)
print(f"Number of separate pages: {len(separate_pages)}")


Number of separate pages: 76


In [320]:

# OyMuPDFLoader loads pages into separate docs!
# This is a problem when we chunk because we only chunk individual
# documents.  We need ONE overall document so that the chunks can
# overlap between actual PDF pages.
document_string = ""
for page in separate_pages:
    document_string += page.page_content
print(f"Length of the document string: {len(document_string)}")


Length of the document string: 189621


In [321]:
import tiktoken

# CHOP IT UP
def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4o").encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap = 200,
    length_function = tiktoken_len
)
text_chunks = text_splitter.split_text(document_string)
print(f"Number of chunks: {len(text_chunks)} ")
max_chunk_size = 0
for chunk in text_chunks:
    max_chunk_size = max(max_chunk_size, len(chunk))
print(f"Maximum chunk size: {max_chunk_size}")
document = [Document(page_content=chunk) for chunk in text_chunks]
print(f"Length of  document: {len(document)}")



Number of chunks: 28 
Maximum chunk size: 9891
Length of  document: 28


In [322]:
from qdrant_client import QdrantClient
import defaults

embedding_model = defaults.default_embedding_model
qdrant_url = defaults.default_url


In [323]:
from qdrant_client import QdrantClient
from qdrant_client.http import models as rest
from langchain_qdrant import QdrantVectorStore
import hashlib

"""
This code creates a hash for every chunk and checks to see if that chunk already exists in the
vector database.  We only want one collection in Qdrant, but want to make sure that if a user
selects a document that has already been embedded and stored, it does not get stored again.  We
also add metadata for the document title, so that we can make our retriever focus on documents of
interest.  For example, after some usage, the application might have 20 documents for the user to 
select from.  We want the retriever to be exactly right for the documents that they selected.

This could also be useful if different versions of documents are in existence.  We would not want to
recreate a large vectorstore.  But the user could select the most recent version.
"""


def get_document_hash(doc_content):
    """Generate a unique hash for the document content."""
    return hashlib.md5(doc_content.encode()).hexdigest()

# Add a unique hash to your documents
for doc in document:
    doc.metadata['content_hash'] = get_document_hash(doc.page_content)

# Add the document title
for doc in document:
    doc.metadata['document_title'] = file_path.split('/')[-1]

client = QdrantClient(url=qdrant_url)

# If the collection exists, then we need to check to see if our document is already
# present, in which case we would not want to store it again.
if client.collection_exists("protocol_collection"):
    print("Collection exists")
    qdrant_vectorstore = QdrantVectorStore.from_existing_collection(
        embedding=embedding_model,
        collection_name="protocol_collection",
        url=qdrant_url
    )
    
    # Check for existing documents and only add new ones
    existing_hashes = set()
    new_docs = []
    
    # Get all existing hashes
    scroll_filter = rest.Filter(
        should=[
            rest.FieldCondition(
                key="metadata.content_hash",
                match=rest.MatchValue(value=doc.metadata['content_hash'])
            ) for doc in document
        ]
    )
    
    scroll_results = client.scroll(
        collection_name="protocol_collection",
        scroll_filter=scroll_filter,
        limit=len(document)  # Adjust this if you have a large number of documents
    )
    
    existing_hashes = set(point.payload.get('metadata', {}).get('content_hash') for point in scroll_results[0])
    
    for doc in document:
        if doc.metadata['content_hash'] not in existing_hashes:
            new_docs.append(doc)
    
    if new_docs:
        qdrant_vectorstore.add_documents(new_docs)
    
    print(f"Added {len(new_docs)} new documents")
    print(f"Skipped {len(existing_hashes)} existing documents")
else: 
    print("Collection does not exist")                           #So we go ahead and just add the documents
    qdrant_vectorstore = QdrantVectorStore.from_documents(
        documents=document,
        embedding=embedding_model,
        collection_name="protocol_collection",
        url=qdrant_url
    )


Collection exists
Added 0 new documents
Skipped 28 existing documents


In [324]:
from qdrant_client.http import models as rest

"""
This code sets up the search type but more importantly it has the filter
set up correctly.  We get a list of document titles that we want to include
in the filter, and pass it into the function, returning the retriever.

"""

def create_protocol_retriever(document_titles):
    return qdrant_vectorstore.as_retriever(
        # search_type='mmr',                                  #mmr is experiment for me
        search_kwargs={
            'filter': rest.Filter(
                must=[
                    rest.FieldCondition(
                        key="metadata.document_title",
                        match=rest.MatchAny(any=document_titles)
                    )
                ]
            ),
            'k': 15,                                         # for the mmr search it will only return k
            # 'fetch_k': 50,                                  # but will evaluate fetch_k candidates
        }
    )

# Usage example
# document_titles = ["consent.pdf", "protocol.pdf"]
document_titles = ["protocol.pdf"]
protocol_retriever = create_protocol_retriever(document_titles)



#### Test protocol retriever

In [325]:
risks = protocol_retriever.get_relevant_documents("Risks of study")
len(risks)
for i, doc in enumerate(risks, 1):
    print(f"Document {i}")
    print("-" * 50)
    print(doc.page_content[0:50])
    print("-" * 50)

Document 1
--------------------------------------------------
blood cell count or anemia. Any blood sampling can
--------------------------------------------------
Document 2
--------------------------------------------------
PRECISE Protocol Version 1.07
Protocol Version Dat
--------------------------------------------------
Document 3
--------------------------------------------------
The Utah IRB has extensive experience with single 
--------------------------------------------------
Document 4
--------------------------------------------------
constitute the CPCCRN Biorepository Governance Com
--------------------------------------------------
Document 5
--------------------------------------------------
10.2.6
Monitoring Serious Adverse Events
A qualifi
--------------------------------------------------
Document 6
--------------------------------------------------
29
4.3.1
Eligibility to Receive Study Drug Doses .
--------------------------------------------------
Document 7
-----

In [326]:
# Check to see how big our prompt context is going to be
total_tokens=0
for risk in risks:
    total_tokens+= tiktoken_len(risk.page_content)
print(f"Tokens in context: {total_tokens}")


Tokens in context: 29294


In [327]:
# Create prompt
from prompts import rag_prompt_template
from langchain.prompts import ChatPromptTemplate
rag_prompt = ChatPromptTemplate.from_template(prompts.rag_prompt_template)

In [328]:
# Create RAG chain
from defaults import default_llm
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser

llm = default_llm

rag_chain = (
    {"context": itemgetter("question") | protocol_retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [329]:

summary_query = """ 
Please write a summary of the protocol that can be used as the introduction to an informed
consent document for a patient to participate in the study described in the protocol.  This
summary should be between 500 and 1000 words in length, and should be understandable by a normally
intelligent adult patient.  You must explain any medical or technical terms in a way
that a high school graduate can understand. The summary should briefly explain the study, the rationale for the study,
the risks and benefits of participation, and that participation is entirely voluntary.

Start the summary with a level 2 Markdown header (##) titled "Study Summary". Then continue with
the content without any further subheadings. Produce the entire summary in Markdown format so it 
can be nicely printed for the reader.

You should assume that the consent form is addressed to the parent of a potentially eligible
child, and the very beginnning of the summary should indicate that they are being invited to allow
their child to participate because the child is potentially eligible. State why their child is potentially
eligible.  

All the details of this introductory summary should be specific for this protocol.

"""

summary = rag_chain.invoke({"question":summary_query})

In [330]:
from IPython.display import display, Markdown
# Display the summary as Markdown
display(Markdown(summary))

## Study Summary

You are being invited to consider allowing your child to participate in a research study because your child has been diagnosed with sepsis-induced multiple organ dysfunction syndrome (MODS) and is receiving care in a Pediatric Intensive Care Unit (PICU) or Cardiac Intensive Care Unit (CICU). This study, called the "PeRsonalizEd immunomodulation in pediatriC sepsIS-inducEd MODS (PRECISE)" study, aims to explore new ways to improve outcomes for children suffering from this serious condition by using personalized treatment approaches.

Sepsis is a life-threatening condition that arises when the body's response to infection causes injury to its own tissues and organs. When sepsis progresses to MODS, it leads to the failure of multiple organs, which significantly increases the risk of morbidity and mortality. Current treatments for sepsis largely focus on antibiotics and supportive care, but there is a need for more targeted therapies that address the specific immune responses in each patient.

The PRECISE study seeks to personalize treatment based on your child's specific immune response to sepsis. Research has shown that children with sepsis-induced MODS can have different immune system responses. Some children may have a weakened immune response, while others may have an overly active immune response. This study will use blood tests to categorize your child's immune response and determine the most appropriate treatment.

There are two main treatment trials in the study: 

1. **GRACE-2 Trial:** This trial is for children whose blood tests show a weakened immune response (immunoparalysis) with mild to moderate inflammation. These children will receive either GM-CSF (a medication that stimulates the immune system) or a placebo.

2. **TRIPS Trial:** This trial is for children who have moderate to severe inflammation without immunoparalysis, or immunoparalysis with severe inflammation. These children will receive either anakinra (a medication that reduces inflammation) or a placebo.

Participation in the study is entirely voluntary. If you choose to allow your child to participate, they will continue to receive standard medical care for sepsis-induced MODS, in addition to the study treatment. The primary goal of the study is to determine whether these personalized treatments can reduce the severity and duration of organ dysfunction.

There are potential risks and benefits to participating in this study. The risks include side effects from the study medications, which are generally mild but can include fever, chills, or allergic reactions. Blood draws for testing may also cause discomfort or bruising. The benefits include close monitoring of your child's condition and the potential for improved outcomes if the study treatments are effective. Additionally, by participating, you will contribute to research that may help improve treatments for other children with sepsis in the future.

It is important to understand that participation is completely voluntary, and you can choose to withdraw your child from the study at any time without affecting their standard medical care. If you decide to participate, you will be asked to sign a consent form, and the study team will provide you with all necessary information and answer any questions you may have.

Thank you for considering this opportunity to contribute to important research that aims to enhance the care and treatment of children with sepsis-induced MODS.

In [331]:
background_query = """ 
Please write a summary of the protocol that can be used as the background section of an informed
consent document for a patient to participate in the study described in the protocol.  This
summary should be between 500 and 1000 words in length, and should be understandable by a normally
intelligent adult patient.  You must explain any medical or technical terms in a way
that a high school graduate can understand. The summary should briefly explain why this patient is being
approached to be in the study, including a brief description of the disease that is being studied in the 
protocol, a description of the study interventions, and the scientific reasons that the investigators believe
the intervention might help the patient.  

Do not include the specific study procedures in this summary, because this will be presented in a different section of 
the informed consent document.  You also do not need to mention that participation is voluntary, nor
the specific risks and benefits of the study, because this information is being presented in a different
part of the informed consent document.

Start the summary with a level 2 Markdown header (##) titled "Background". Then continue with
the content without any further subheadings. Produce the entire summary in Markdown format so it 
can be nicely printed for the reader. 

All the details of this background summary should be specific for this protocol.

"""

background = rag_chain.invoke({"question":background_query})

display(Markdown(background))

## Background

You are being invited to participate in a research study focused on improving the treatment of children with sepsis-induced multiple organ dysfunction syndrome (MODS). Sepsis is a serious medical condition that occurs when the body's response to an infection injures its own tissues and organs. This can lead to multiple organ dysfunction syndrome, where two or more organ systems in the body stop working properly. MODS is a leading cause of morbidity and mortality in critically ill children, and current treatments are mostly supportive, such as antibiotics and medications to support blood pressure.

The study is called "PeRsonalizEd immunomodulation in pediatriC sepsIS-inducEd MODS" (PRECISE). The goal of this study is to test new treatments that are personalized to a child's specific immune response to sepsis. Researchers believe that by tailoring the treatment to fit the unique immune system response of each child, they can improve outcomes and reduce the severity and duration of organ dysfunction.

In this study, children with sepsis-induced MODS will be grouped into different categories based on their immune system's response to the infection. The categories include those with "immunoparalysis," where the immune system is not responding strongly enough, and those with "hyperinflammation," where the immune system is overactive.

For children with immunoparalysis, the study will test a drug called GM-CSF (granulocyte-macrophage colony-stimulating factor). GM-CSF is a natural protein in the body that helps stimulate the immune system. The researchers believe that GM-CSF could help boost the immune response in children with immunoparalysis, potentially leading to faster recovery.

For children with hyperinflammation, the study will test a drug called anakinra. Anakinra is designed to reduce inflammation by blocking a specific protein in the immune system called interleukin-1. By dampening the overactive immune response, anakinra may help reduce the damage to organs caused by the inflammation.

The scientific rationale behind these treatments is based on previous research that shows children with sepsis-induced MODS often have very different immune responses. Some children’s immune systems are underactive, while others are overactive. By identifying these different responses and treating them accordingly, the researchers hope to improve outcomes for children with this serious condition.

Your participation will help the researchers understand whether these targeted treatments can effectively improve recovery in children with sepsis-induced MODS. The findings from this study may lead to more personalized and effective treatments for this condition in the future, potentially helping many children who suffer from severe sepsis.

In [332]:
number_of_participants_query = """ 
Please write a summary of the protocol that can be used 
for the "number of participants" section of the informed consent document.  This should include where the
study is being conducted (for example, at this hospital, or in a network, or in multiple hospitals), the funding source
(often the NIH), the total number of participants that are planned to be enrolled in the study,
and the total period of the time that the study is expected to enroll subjects. This summary should not require more than 200 words.

Start the summary with a level 2 Markdown header (##) titled "Number of Participants". Then continue with
the content without any further subheadings. Produce the entire summary in Markdown format so it 
can be nicely printed for the reader. 

All the details of this number of participants summary should be specific for this protocol.

"""

number_of_participants = rag_chain.invoke({"question":number_of_participants_query})

display(Markdown(number_of_participants))

## Number of Participants

This study is being conducted by the Collaborative Pediatric Critical Care Research Network (CPCCRN) across multiple hospitals, including children's hospitals and intensive care units within large academic health centers. The research is funded by the Eunice Kennedy Shriver National Institute for Child Health and Human Development (NICHD). The protocol aims to enroll up to 1,095 participants in total. Specifically, 500 subjects are expected to participate in the TRIPS trial, and 400 subjects in the GRACE-2 trial. The study will include infants, children, and adolescent patients admitted with sepsis-induced multiple organ dysfunction syndrome (MODS). It is anticipated that enrollment will take place over a four-year period. Participants will be identified and recruited by on-site study staff at the participating hospitals, with eligibility determined based on specific inclusion and exclusion criteria related to their clinical condition. The study aims to assess the effects of personalized immune modulation on reducing mortality and morbidity in pediatric patients with sepsis-induced MODS.

In [333]:
study_procedures_query = """ 
Please write a detailed summary of all the study procedures that will be carried out in this protocol.  This will
be used for the "study procedures" section of the informed consent document that the patient will read when deciding
whether to participate in the study, so it is important that all significant procedures be included.  
Make sure that everything will be understandable to the reader, who is not trained in medicine.  Do not write
the summary as if it is in third person - write it like you are speaking directly to the patient (i.e. use "you" instead 
of the "patient", with correct grammar of course.)  Do not include a welcome to the study, or discussion about
participation being voluntary, as that information is in a different part of the consent document.  Do not include
risks and benefits as these are presented in a different part of the consent document.  Please be detailed, as it is 
important that the patient understand each procedure.
The length of this summary is usually
2000 to 3000 words.

Start the summary with a level 2 Markdown header (##) titled "Study Procedures", and then continue the section with subheadings
that will help organize the information for the reader.  Do not go more than two subheadings deep.

All the details of study procedures should be specific to this protocol.

"""

study_procedures = rag_chain.invoke({"question":study_procedures_query})
display(Markdown(study_procedures))

## Study Procedures

### Screening and Enrollment

When you are admitted to the Pediatric or Cardiac Intensive Care Unit, our research staff will check if you meet the criteria for participation in this study. This involves reviewing your medical records to ensure you have a documented or suspected infection that has led to the onset of multiple organ dysfunction syndrome (MODS). If you meet the criteria, a member of our team will discuss the study with your guardian(s) and ask for permission to proceed with enrollment.

### Immune Phenotyping

Immune phenotyping is a crucial part of the study, which helps us understand how your immune system is responding to the infection that caused MODS. On the second day after MODS is identified, we will draw approximately 2 milliliters of blood. This blood sample will be used to measure your immune response and levels of certain inflammatory markers, like TNFα, ferritin, and CRP. These measurements will help determine the best course of treatment for you within the study.

### Randomization into Interventional Trials

Based on the results from immune phenotyping, you may be eligible for one of two interventional trials: the GRACE-2 trial or the TRIPS trial. The GRACE-2 trial involves the administration of GM-CSF (Granulocyte Macrophage-Colony Stimulating Factor) or a placebo, while the TRIPS trial involves treatment with anakinra or a placebo. Randomization means that you will be assigned to one of these groups by chance, similar to flipping a coin, to ensure the study results are unbiased.

### GRACE-2 Study Drug Administration

If you are randomized into the GRACE-2 trial, you will receive the study drug or placebo intravenously. The drug is infused over at least six hours each day for up to seven days. We will closely monitor your white blood cell count during this time. If your white blood cell count rises above a certain level, we may temporarily stop the infusion to ensure your safety.

### TRIPS Study Drug Administration

For participants in the TRIPS trial, anakinra or a placebo will be administered every 12 hours through an intravenous line, with each dose taking about an hour to infuse. This treatment will continue for seven days. We will adjust the dose for participants with severe kidney dysfunction to ensure it is safe and effective.

### Follow-up Immune Phenotyping

Throughout the study, we will continue to monitor your immune system and inflammation levels. Additional blood samples will be collected twice a week for up to two weeks to track changes in your immune response. These follow-up tests will provide us with valuable information on how the treatment is affecting your immune system.

### Biorepository Sample Collection and Processing

In addition to the regular immune phenotyping, we will collect small amounts of extra blood and, if applicable, endotracheal aspirates for a biorepository. These samples will be stored for future research related to the study to help us better understand sepsis-induced MODS and its treatment. Your guardian(s) will be asked for permission to store these samples, and they can decide whether they want to allow this.

### Discontinuation of Study Drug

If at any point during the study you experience adverse events that require stopping the study drug, we will discontinue the treatment. However, this does not mean you will be withdrawn from the study. We will continue to collect data and samples as part of the study procedures unless your guardian(s) request complete withdrawal from the study.

### Monitoring and Data Collection

We will monitor your health closely throughout the study. This includes daily assessments of your organ function and any changes in your condition. We will collect data on your health status, treatments you receive, and any side effects you experience. This information will help us understand the effects of the study drugs and improve treatments for others in the future.

### Conclusion of Study Participation

Once the study period is over, we may continue to follow up with you to understand the long-term effects of the treatments. This follow-up includes assessments at three and twelve months after the initial treatment period. We will use this information to evaluate the overall impact of the treatments on your health and quality of life.

By participating in this study, you are contributing to important research that aims to improve the treatment and outcomes of children with sepsis-induced MODS. Thank you for considering being a part of this study.

In [334]:
alt_procedures_query = """ 
Please write a  summary of alternatives to participation in this study.  An example is:

" Your participation in this study is voluntary.  It is not necessary to be in this study to get care for
your illness.  Monitoring of immune function is not currently done as part of routine ICU care.  There are no
other treatments designed to increase immune function of reduce inflammation that are routinely used in 
children with sepsis."

Note that this example is purely an example, and your summary must be specific to the protocol. The summary should
be easily understandable by medically untrained readers.  This section is usually less than 500 words in length.


Start the summary with a level 2 Markdown header (##) titled "Alternative Procedures", and then continue with
the content without any further subheadings. Produce the entire summary in Markdown format so it 
can be nicely printed for the reader. 


"""

alt_procedures = rag_chain.invoke({"question":alt_procedures_query})
display(Markdown(alt_procedures))

## Alternative Procedures

Your participation in this study is entirely voluntary. It is not required to be part of this study to receive high-quality medical care for sepsis-induced multiple organ dysfunction syndrome (MODS). All children will continue to receive state-of-the-art critical care as per standard medical practices, regardless of their participation. This includes comprehensive support measures such as antibiotics, fluid resuscitation, and other treatments as determined by their clinical team. Monitoring immune function and providing personalized immunomodulatory treatments, as proposed in this study, are not currently part of routine clinical practice. Therefore, no alternative treatments focused on adjusting or enhancing immune function specifically for sepsis-induced MODS are available outside this study. Parents can choose to discontinue their child’s participation in the study at any time, and any data collected up to that point will still contribute to the research.

In [335]:
risks_query = """ 
Please write a detailed summary of the risks of participating in  the study.  This will be used for the
"Risks" section of the informed consent document.  It is important that all significant risks of study
participation are described in detail. The summary must be easily readable by untrained readers, so provide
definitions of technical or medical terms.  Address all the risks by speaking to the patient, not by passively
mentioning risks to "the patient".  Especially include risks that are associated with the study interventions such
as drugs or devices, or associated with testing that is done as part of the study.  Also include
the risks associated with data collection, and also mention "unforeseable risks".

The length of this risk summary is usually
2000 to 3000 words.

Start the summary with a level 2 Markdown header (##) titled "Risks", and then continue the section with subheadings
that will help organize the information for the reader.  Do not go more than two subheadings deep.

All the details of study risks should be specific to this protocol.

"""

risks = rag_chain.invoke({"question":risks_query})
display(Markdown(risks))

## Risks

### Introduction

Participating in a clinical study can provide valuable insights and potentially lead to improved treatment strategies. However, it's essential to be aware of the risks involved in such studies. This summary will guide you through the potential risks associated with participating in this study, including those related to the study drugs, procedures, and data collection. We aim to explain these risks in a way that is easy to understand, even if you are not familiar with medical terms.

### Risks Associated with Study Drugs

#### GM-CSF (Granulocyte Macrophage-Colony Stimulating Factor)

GM-CSF is a medication used in this study to help boost your immune system. While it can be beneficial, it also carries certain risks:

- **Fever and Chills**: You might experience an increase in body temperature and shivering.
- **Bone Pain**: Some participants report discomfort in their bones.
- **Dyspnea (Difficulty Breathing)**: Breathing may become more laborious.
- **Tachycardia (Rapid Heartbeat)**: Your heart might beat faster than usual.
- **Peripheral Edema (Swelling of the Limbs)**: Swelling can occur, particularly if the drug is infused too quickly. However, we administer it slowly to minimize this risk.
- **Allergic Reactions**: Although rare, some participants may have an allergic response to the drug.

#### Anakinra

Anakinra is another medication used in this study to manage inflammation. Its use is generally safe, but it is not without risks:

- **Increased Risk of Infections**: Anakinra may slightly increase your risk of infections due to its effect on the immune system.
- **Neutropenia (Low White Blood Cell Count)**: This condition, which involves a reduced ability to fight infections, is less likely due to the short duration of treatment in this study.
- **Hypersensitivity Reactions**: Although rare, severe allergic reactions can occur.

### Risks Associated with Study Procedures

#### Blood Sampling

As part of the study, blood samples will be collected to monitor your health and the effects of the medications:

- **Pain and Bruising**: Blood draws can cause discomfort and bruising at the site of the needle insertion.
- **Infection**: There is a very small risk of infection where the needle enters the skin.
- **Anemia**: Frequent blood sampling may lead to a decrease in your red blood cell count, which can result in anemia, making you feel tired and weak.

#### Venipuncture and Capillary Sticks

Additional blood samples may be required, which could involve:

- **Pain and Discomfort**: These procedures can cause temporary discomfort.
- **Bleeding and Infection**: There's a minor risk of bleeding and infection at the puncture site.

### Risks Associated with Data Collection

#### Loss of Confidentiality

While every effort is made to ensure your personal information remains confidential, there is a minimal risk that your data could be accidentally disclosed. This study uses advanced security measures to protect your information, but no system is infallible.

### Unforeseeable Risks

It's important to understand that, while we have outlined the known risks, there may be unforeseen risks or side effects that arise during the study. These are risks that are not currently known but could potentially occur.

### Conclusion

Participation in this study involves certain risks, as outlined above. It's crucial to weigh these risks against the potential benefits of contributing to medical research. We are committed to minimizing risks and ensuring your safety throughout the study. If you have any concerns or questions about the risks, please feel free to discuss them with the study team. Your participation is voluntary, and you can withdraw from the study at any time without affecting your standard medical care.

In [336]:
benefits_query = """ 
Please write a  summary of the potential benefits of participating in  the study.  This will be used for the
"Benefits" section of the informed consent document.  The summary should include potential benefits for the patient
(addressed as "you"), and potential benefits for others.  Since this is a research study and it is
not known if the intervention is helpful, it is important to not overstate potential benefits for the patient.

The length of this risk summary is usually
500 to 750 words.

Start the summary with a level 2 Markdown header (##) titled "Benefits",  and then continue 
with a subheading for "Potential
Benefits for You" and another subheading for "Potential Benefits for Others".

All the information of study benefits should be specific to this protocol.

"""

benefits = rag_chain.invoke({"question":benefits_query})
display(Markdown(benefits))

## Benefits

### Potential Benefits for You

As a participant in this research study focused on pediatric sepsis-induced multiple organ dysfunction syndrome (MODS), you may receive personalized treatment that targets specific immune responses. This approach aims to improve your recovery from sepsis-induced MODS by enhancing your immune function and potentially accelerating the healing process. If you are selected for one of the intervention groups, you will receive either GM-CSF or anakinra, based on your immune profile. These treatments are intended to help normalize immune function, facilitate infection clearance, and promote tissue healing, which may lead to a quicker resolution of organ failure.

It is important to understand that this is a research study, and while these interventions have shown promise in previous studies, it is not yet confirmed if they will be effective for you. The primary goal is to assess the effectiveness of these treatments in reducing the severity and duration of organ dysfunction.

### Potential Benefits for Others

Your participation in this study could contribute significantly to the advancement of medical knowledge and the development of new treatment strategies for pediatric sepsis-induced MODS. By participating, you are helping researchers gather important data that may lead to more effective and personalized treatments for children suffering from similar conditions in the future.

The insights gained from this research could potentially reduce mortality and improve outcomes for critically ill children by identifying and validating new therapeutic approaches. This study aims to establish whether personalized immunomodulation can be a beneficial strategy, potentially influencing future clinical practices and improving the standard of care for pediatric patients with sepsis-induced MODS.

Additionally, the study will provide valuable information about the immune responses in sepsis-induced MODS, which could help in understanding the disease better and tailoring treatments for individual patients based on their unique immune profiles. This personalized approach could pave the way for further research and development of targeted therapies, ultimately benefiting a wider pediatric population beyond the current study participants.

In [337]:
from docx import Document
from docx.shared import Pt
from docx.enum.style import WD_STYLE_TYPE
from docx.shared import Inches
import re
from IPython.display import display, Markdown

# ... (previous code for combining summaries remains the same)

# Create a new Word document
doc = Document()

# Define styles
styles = doc.styles
if 'Normal' not in styles:
    style_normal = styles.add_style('Normal', WD_STYLE_TYPE.PARAGRAPH)
    style_normal.font.size = Pt(11)
    style_normal.paragraph_format.space_after = Pt(0)
    style_normal.paragraph_format.space_before = Pt(0)

if 'Heading 1' not in styles:
    style_heading1 = styles.add_style('Heading 1', WD_STYLE_TYPE.PARAGRAPH)
    style_heading1.font.size = Pt(16)
    style_heading1.font.bold = True
    style_heading1.paragraph_format.space_before = Pt(6)
    style_heading1.paragraph_format.space_after = Pt(6)

if 'Heading 2' not in styles:
    style_heading2 = styles.add_style('Heading 2', WD_STYLE_TYPE.PARAGRAPH)
    style_heading2.font.size = Pt(14)
    style_heading2.font.bold = True
    style_heading2.paragraph_format.space_before = Pt(6)
    style_heading2.paragraph_format.space_after = Pt(6)

if 'Heading 3' not in styles:
    style_heading3 = styles.add_style('Heading 3', WD_STYLE_TYPE.PARAGRAPH)
    style_heading3.font.size = Pt(12)
    style_heading3.font.bold = True
    style_heading3.paragraph_format.space_before = Pt(6)
    style_heading3.paragraph_format.space_after = Pt(6)

if 'Heading 4' not in styles:
    style_heading4 = styles.add_style('Heading 4', WD_STYLE_TYPE.PARAGRAPH)
    style_heading4.font.size = Pt(11)
    style_heading4.font.bold = True
    style_heading4.font.italic = True
    style_heading4.paragraph_format.space_before = Pt(6)
    style_heading4.paragraph_format.space_after = Pt(6)

def add_formatted_text(paragraph, text):
    parts = re.split(r'(\*\*.*?\*\*|\*.*?\*)', text)
    for part in parts:
        if part.startswith('**') and part.endswith('**'):
            run = paragraph.add_run(part[2:-2])
            run.bold = True
        elif part.startswith('*') and part.endswith('*'):
            run = paragraph.add_run(part[1:-1])
            run.italic = True
        else:
            paragraph.add_run(part)

# Add content to the document
doc.add_heading('Full Informed Consent Document', 0)

for section in summaries:
    lines = section.split('\n')
    for line in lines:
        if line.startswith('#### '):
            p = doc.add_paragraph(line[5:], style='Heading 4')
        elif line.startswith('### '):
            p = doc.add_paragraph(line[4:], style='Heading 3')
        elif line.startswith('## '):
            p = doc.add_paragraph(line[3:], style='Heading 2')
        elif line.startswith('# '):
            p = doc.add_paragraph(line[2:], style='Heading 1')
        elif line.strip().startswith('- '):
            p = doc.add_paragraph(line.strip()[2:], style='List Bullet')
        elif line.strip() == '---':
            p = doc.add_paragraph('', style='Normal')
            p.paragraph_format.space_after = Pt(6)
        else:
            p = doc.add_paragraph(style='Normal')
            add_formatted_text(p, line)
        
        # Ensure consistent spacing for all paragraphs
        if p.style.name == 'Normal':
            p.paragraph_format.space_after = Pt(0)
            p.paragraph_format.space_before = Pt(0)

# Save the document
doc.save('informed_consent_document.docx')

print("Word document has been created: informed_consent_document.docx")

Word document has been created: informed_consent_document.docx


In [338]:
from IPython.display import display, Markdown
import os

# Assuming you have already generated these summaries
summaries = [
    summary,
    background,
    number_of_participants,
    study_procedures,
    alt_procedures,
    risks,
    benefits
]

# Combine all summaries into one string
combined_markdown = "\n\n---\n\n".join(summaries)

# Add a title to the combined document
full_document = "# Full Informed Consent Document\n\n" + combined_markdown

# Display the full document in the notebook
display(Markdown(full_document))

# Save the Markdown content to a file
file_path = 'informed_consent_document.md'
with open(file_path, 'w', encoding='utf-8') as f:
    f.write(full_document)

print(f"Markdown document has been created: {os.path.abspath(file_path)}")

# Full Informed Consent Document

## Study Summary

You are being invited to consider allowing your child to participate in a research study because your child has been diagnosed with sepsis-induced multiple organ dysfunction syndrome (MODS) and is receiving care in a Pediatric Intensive Care Unit (PICU) or Cardiac Intensive Care Unit (CICU). This study, called the "PeRsonalizEd immunomodulation in pediatriC sepsIS-inducEd MODS (PRECISE)" study, aims to explore new ways to improve outcomes for children suffering from this serious condition by using personalized treatment approaches.

Sepsis is a life-threatening condition that arises when the body's response to infection causes injury to its own tissues and organs. When sepsis progresses to MODS, it leads to the failure of multiple organs, which significantly increases the risk of morbidity and mortality. Current treatments for sepsis largely focus on antibiotics and supportive care, but there is a need for more targeted therapies that address the specific immune responses in each patient.

The PRECISE study seeks to personalize treatment based on your child's specific immune response to sepsis. Research has shown that children with sepsis-induced MODS can have different immune system responses. Some children may have a weakened immune response, while others may have an overly active immune response. This study will use blood tests to categorize your child's immune response and determine the most appropriate treatment.

There are two main treatment trials in the study: 

1. **GRACE-2 Trial:** This trial is for children whose blood tests show a weakened immune response (immunoparalysis) with mild to moderate inflammation. These children will receive either GM-CSF (a medication that stimulates the immune system) or a placebo.

2. **TRIPS Trial:** This trial is for children who have moderate to severe inflammation without immunoparalysis, or immunoparalysis with severe inflammation. These children will receive either anakinra (a medication that reduces inflammation) or a placebo.

Participation in the study is entirely voluntary. If you choose to allow your child to participate, they will continue to receive standard medical care for sepsis-induced MODS, in addition to the study treatment. The primary goal of the study is to determine whether these personalized treatments can reduce the severity and duration of organ dysfunction.

There are potential risks and benefits to participating in this study. The risks include side effects from the study medications, which are generally mild but can include fever, chills, or allergic reactions. Blood draws for testing may also cause discomfort or bruising. The benefits include close monitoring of your child's condition and the potential for improved outcomes if the study treatments are effective. Additionally, by participating, you will contribute to research that may help improve treatments for other children with sepsis in the future.

It is important to understand that participation is completely voluntary, and you can choose to withdraw your child from the study at any time without affecting their standard medical care. If you decide to participate, you will be asked to sign a consent form, and the study team will provide you with all necessary information and answer any questions you may have.

Thank you for considering this opportunity to contribute to important research that aims to enhance the care and treatment of children with sepsis-induced MODS.

---

## Background

You are being invited to participate in a research study focused on improving the treatment of children with sepsis-induced multiple organ dysfunction syndrome (MODS). Sepsis is a serious medical condition that occurs when the body's response to an infection injures its own tissues and organs. This can lead to multiple organ dysfunction syndrome, where two or more organ systems in the body stop working properly. MODS is a leading cause of morbidity and mortality in critically ill children, and current treatments are mostly supportive, such as antibiotics and medications to support blood pressure.

The study is called "PeRsonalizEd immunomodulation in pediatriC sepsIS-inducEd MODS" (PRECISE). The goal of this study is to test new treatments that are personalized to a child's specific immune response to sepsis. Researchers believe that by tailoring the treatment to fit the unique immune system response of each child, they can improve outcomes and reduce the severity and duration of organ dysfunction.

In this study, children with sepsis-induced MODS will be grouped into different categories based on their immune system's response to the infection. The categories include those with "immunoparalysis," where the immune system is not responding strongly enough, and those with "hyperinflammation," where the immune system is overactive.

For children with immunoparalysis, the study will test a drug called GM-CSF (granulocyte-macrophage colony-stimulating factor). GM-CSF is a natural protein in the body that helps stimulate the immune system. The researchers believe that GM-CSF could help boost the immune response in children with immunoparalysis, potentially leading to faster recovery.

For children with hyperinflammation, the study will test a drug called anakinra. Anakinra is designed to reduce inflammation by blocking a specific protein in the immune system called interleukin-1. By dampening the overactive immune response, anakinra may help reduce the damage to organs caused by the inflammation.

The scientific rationale behind these treatments is based on previous research that shows children with sepsis-induced MODS often have very different immune responses. Some children’s immune systems are underactive, while others are overactive. By identifying these different responses and treating them accordingly, the researchers hope to improve outcomes for children with this serious condition.

Your participation will help the researchers understand whether these targeted treatments can effectively improve recovery in children with sepsis-induced MODS. The findings from this study may lead to more personalized and effective treatments for this condition in the future, potentially helping many children who suffer from severe sepsis.

---

## Number of Participants

This study is being conducted by the Collaborative Pediatric Critical Care Research Network (CPCCRN) across multiple hospitals, including children's hospitals and intensive care units within large academic health centers. The research is funded by the Eunice Kennedy Shriver National Institute for Child Health and Human Development (NICHD). The protocol aims to enroll up to 1,095 participants in total. Specifically, 500 subjects are expected to participate in the TRIPS trial, and 400 subjects in the GRACE-2 trial. The study will include infants, children, and adolescent patients admitted with sepsis-induced multiple organ dysfunction syndrome (MODS). It is anticipated that enrollment will take place over a four-year period. Participants will be identified and recruited by on-site study staff at the participating hospitals, with eligibility determined based on specific inclusion and exclusion criteria related to their clinical condition. The study aims to assess the effects of personalized immune modulation on reducing mortality and morbidity in pediatric patients with sepsis-induced MODS.

---

## Study Procedures

### Screening and Enrollment

When you are admitted to the Pediatric or Cardiac Intensive Care Unit, our research staff will check if you meet the criteria for participation in this study. This involves reviewing your medical records to ensure you have a documented or suspected infection that has led to the onset of multiple organ dysfunction syndrome (MODS). If you meet the criteria, a member of our team will discuss the study with your guardian(s) and ask for permission to proceed with enrollment.

### Immune Phenotyping

Immune phenotyping is a crucial part of the study, which helps us understand how your immune system is responding to the infection that caused MODS. On the second day after MODS is identified, we will draw approximately 2 milliliters of blood. This blood sample will be used to measure your immune response and levels of certain inflammatory markers, like TNFα, ferritin, and CRP. These measurements will help determine the best course of treatment for you within the study.

### Randomization into Interventional Trials

Based on the results from immune phenotyping, you may be eligible for one of two interventional trials: the GRACE-2 trial or the TRIPS trial. The GRACE-2 trial involves the administration of GM-CSF (Granulocyte Macrophage-Colony Stimulating Factor) or a placebo, while the TRIPS trial involves treatment with anakinra or a placebo. Randomization means that you will be assigned to one of these groups by chance, similar to flipping a coin, to ensure the study results are unbiased.

### GRACE-2 Study Drug Administration

If you are randomized into the GRACE-2 trial, you will receive the study drug or placebo intravenously. The drug is infused over at least six hours each day for up to seven days. We will closely monitor your white blood cell count during this time. If your white blood cell count rises above a certain level, we may temporarily stop the infusion to ensure your safety.

### TRIPS Study Drug Administration

For participants in the TRIPS trial, anakinra or a placebo will be administered every 12 hours through an intravenous line, with each dose taking about an hour to infuse. This treatment will continue for seven days. We will adjust the dose for participants with severe kidney dysfunction to ensure it is safe and effective.

### Follow-up Immune Phenotyping

Throughout the study, we will continue to monitor your immune system and inflammation levels. Additional blood samples will be collected twice a week for up to two weeks to track changes in your immune response. These follow-up tests will provide us with valuable information on how the treatment is affecting your immune system.

### Biorepository Sample Collection and Processing

In addition to the regular immune phenotyping, we will collect small amounts of extra blood and, if applicable, endotracheal aspirates for a biorepository. These samples will be stored for future research related to the study to help us better understand sepsis-induced MODS and its treatment. Your guardian(s) will be asked for permission to store these samples, and they can decide whether they want to allow this.

### Discontinuation of Study Drug

If at any point during the study you experience adverse events that require stopping the study drug, we will discontinue the treatment. However, this does not mean you will be withdrawn from the study. We will continue to collect data and samples as part of the study procedures unless your guardian(s) request complete withdrawal from the study.

### Monitoring and Data Collection

We will monitor your health closely throughout the study. This includes daily assessments of your organ function and any changes in your condition. We will collect data on your health status, treatments you receive, and any side effects you experience. This information will help us understand the effects of the study drugs and improve treatments for others in the future.

### Conclusion of Study Participation

Once the study period is over, we may continue to follow up with you to understand the long-term effects of the treatments. This follow-up includes assessments at three and twelve months after the initial treatment period. We will use this information to evaluate the overall impact of the treatments on your health and quality of life.

By participating in this study, you are contributing to important research that aims to improve the treatment and outcomes of children with sepsis-induced MODS. Thank you for considering being a part of this study.

---

## Alternative Procedures

Your participation in this study is entirely voluntary. It is not required to be part of this study to receive high-quality medical care for sepsis-induced multiple organ dysfunction syndrome (MODS). All children will continue to receive state-of-the-art critical care as per standard medical practices, regardless of their participation. This includes comprehensive support measures such as antibiotics, fluid resuscitation, and other treatments as determined by their clinical team. Monitoring immune function and providing personalized immunomodulatory treatments, as proposed in this study, are not currently part of routine clinical practice. Therefore, no alternative treatments focused on adjusting or enhancing immune function specifically for sepsis-induced MODS are available outside this study. Parents can choose to discontinue their child’s participation in the study at any time, and any data collected up to that point will still contribute to the research.

---

## Risks

### Introduction

Participating in a clinical study can provide valuable insights and potentially lead to improved treatment strategies. However, it's essential to be aware of the risks involved in such studies. This summary will guide you through the potential risks associated with participating in this study, including those related to the study drugs, procedures, and data collection. We aim to explain these risks in a way that is easy to understand, even if you are not familiar with medical terms.

### Risks Associated with Study Drugs

#### GM-CSF (Granulocyte Macrophage-Colony Stimulating Factor)

GM-CSF is a medication used in this study to help boost your immune system. While it can be beneficial, it also carries certain risks:

- **Fever and Chills**: You might experience an increase in body temperature and shivering.
- **Bone Pain**: Some participants report discomfort in their bones.
- **Dyspnea (Difficulty Breathing)**: Breathing may become more laborious.
- **Tachycardia (Rapid Heartbeat)**: Your heart might beat faster than usual.
- **Peripheral Edema (Swelling of the Limbs)**: Swelling can occur, particularly if the drug is infused too quickly. However, we administer it slowly to minimize this risk.
- **Allergic Reactions**: Although rare, some participants may have an allergic response to the drug.

#### Anakinra

Anakinra is another medication used in this study to manage inflammation. Its use is generally safe, but it is not without risks:

- **Increased Risk of Infections**: Anakinra may slightly increase your risk of infections due to its effect on the immune system.
- **Neutropenia (Low White Blood Cell Count)**: This condition, which involves a reduced ability to fight infections, is less likely due to the short duration of treatment in this study.
- **Hypersensitivity Reactions**: Although rare, severe allergic reactions can occur.

### Risks Associated with Study Procedures

#### Blood Sampling

As part of the study, blood samples will be collected to monitor your health and the effects of the medications:

- **Pain and Bruising**: Blood draws can cause discomfort and bruising at the site of the needle insertion.
- **Infection**: There is a very small risk of infection where the needle enters the skin.
- **Anemia**: Frequent blood sampling may lead to a decrease in your red blood cell count, which can result in anemia, making you feel tired and weak.

#### Venipuncture and Capillary Sticks

Additional blood samples may be required, which could involve:

- **Pain and Discomfort**: These procedures can cause temporary discomfort.
- **Bleeding and Infection**: There's a minor risk of bleeding and infection at the puncture site.

### Risks Associated with Data Collection

#### Loss of Confidentiality

While every effort is made to ensure your personal information remains confidential, there is a minimal risk that your data could be accidentally disclosed. This study uses advanced security measures to protect your information, but no system is infallible.

### Unforeseeable Risks

It's important to understand that, while we have outlined the known risks, there may be unforeseen risks or side effects that arise during the study. These are risks that are not currently known but could potentially occur.

### Conclusion

Participation in this study involves certain risks, as outlined above. It's crucial to weigh these risks against the potential benefits of contributing to medical research. We are committed to minimizing risks and ensuring your safety throughout the study. If you have any concerns or questions about the risks, please feel free to discuss them with the study team. Your participation is voluntary, and you can withdraw from the study at any time without affecting your standard medical care.

---

## Benefits

### Potential Benefits for You

As a participant in this research study focused on pediatric sepsis-induced multiple organ dysfunction syndrome (MODS), you may receive personalized treatment that targets specific immune responses. This approach aims to improve your recovery from sepsis-induced MODS by enhancing your immune function and potentially accelerating the healing process. If you are selected for one of the intervention groups, you will receive either GM-CSF or anakinra, based on your immune profile. These treatments are intended to help normalize immune function, facilitate infection clearance, and promote tissue healing, which may lead to a quicker resolution of organ failure.

It is important to understand that this is a research study, and while these interventions have shown promise in previous studies, it is not yet confirmed if they will be effective for you. The primary goal is to assess the effectiveness of these treatments in reducing the severity and duration of organ dysfunction.

### Potential Benefits for Others

Your participation in this study could contribute significantly to the advancement of medical knowledge and the development of new treatment strategies for pediatric sepsis-induced MODS. By participating, you are helping researchers gather important data that may lead to more effective and personalized treatments for children suffering from similar conditions in the future.

The insights gained from this research could potentially reduce mortality and improve outcomes for critically ill children by identifying and validating new therapeutic approaches. This study aims to establish whether personalized immunomodulation can be a beneficial strategy, potentially influencing future clinical practices and improving the standard of care for pediatric patients with sepsis-induced MODS.

Additionally, the study will provide valuable information about the immune responses in sepsis-induced MODS, which could help in understanding the disease better and tailoring treatments for individual patients based on their unique immune profiles. This personalized approach could pave the way for further research and development of targeted therapies, ultimately benefiting a wider pediatric population beyond the current study participants.

Markdown document has been created: /Users/jmichaeldean/clinical-trials/experimental/informed_consent_document.md
